In [2]:
import os
import pandas as pd
from openpyxl import load_workbook

In [3]:
# Define the path to the Excel file
folder_path = "../BoS (Business Objects) Raw Data Reports - Deidentified"
file_name = "143 RAW Client Data Export v3_EE Workflow.xlsx"
file_path = os.path.join(folder_path, file_name)

In [4]:
# Get the program from the file name
program_name = file_name.split(" ")[0]
print(f"We are looking at Program {program_name} data")

We are looking at Program 143 data


In [5]:
entry_exit = pd.read_excel(file_path, sheet_name="ENTRY-EXIT")
entry_exit.head()

,EE Provider ID,Entry Date,Housing Move-in Date(12855),Exit Date,Destination,Reason for Leaving,Entry Exit Type,Entry Exit Date Added,Entry Exit User Creating,Entry Exit Date Updated,Entry Exit User Updating
0,Community Housing Network - Oakland County - R...,2024-08-26 02:00:00,2024-10-18 12:00:00,NaT,NaN,NaN,HUD,2024-08-26 14:39:21.299,Nicole Hawkins(17760),2024-08-26 14:39:21.407,Nicole Hawkins(17760)
1,Community Housing Network - Oakland County - R...,2024-08-26 02:00:00,2024-10-18 12:00:00,NaT,NaN,NaN,HUD,2024-08-26 14:39:21.421,Nicole Hawkins(17760),2024-08-26 14:39:21.421,Nicole Hawkins(17760)
2,Community Housing Network - Oakland County - R...,2024-08-26 02:00:00,2024-10-18 12:00:00,NaT,NaN,NaN,HUD,2024-08-26 14:39:21.471,Nicole Hawkins(17760),2024-08-26 14:39:21.471,Nicole Hawkins(17760)
3,Community Housing Network - Oakland County - R...,2024-08-16 02:00:00,2024-10-17 12:00:00,NaT,NaN,NaN,HUD,2024-08-19 15:24:07.108,Nicole Hawkins(17760),2024-08-19 15:24:07.108,Nicole Hawkins(17760)
4,Community Housing Network - Oakland County - R...,2024-06-10 02:00:00,2024-10-16 12:00:00,NaT,NaN,NaN,HUD,2024-09-25 11:14:20.994,Nicole Hawkins(17760),2024-09-25 11:14:20.994,Nicole Hawkins(17760)


In [12]:
# Convert to datetime
entry_exit['Entry Date'] = pd.to_datetime(entry_exit['Entry Date'])
entry_exit['Exit Date'] = pd.to_datetime(entry_exit['Exit Date'])

# Fill missing Exit Date with current date (without time)
today = pd.Timestamp.today().normalize()
entry_exit['Exit Date'] = entry_exit['Exit Date'].fillna(today)

# Create 'type' column
entry_exit['type'] = entry_exit['Exit Date'].eq(today).map({True: 'Stayers', False: 'Leavers'})

# Calculate 'Stay of length' in days
entry_exit['Stay of length'] = (entry_exit['Exit Date'] - entry_exit['Entry Date']).dt.days

# Keep only the 4 relevant columns
entry_exit = entry_exit[['Entry Date', 'Exit Date', 'type', 'Stay of length']]

entry_exit.head()

,Entry Date,Exit Date,type,Stay of length
0,2024-08-26 02:00:00,2025-04-10,Stayers,226
1,2024-08-26 02:00:00,2025-04-10,Stayers,226
2,2024-08-26 02:00:00,2025-04-10,Stayers,226
3,2024-08-16 02:00:00,2025-04-10,Stayers,236
4,2024-06-10 02:00:00,2025-04-10,Stayers,303


In [14]:
# Group by 'type' and aggregate
summary_df = entry_exit.groupby('type')['Stay of length'].agg(
    mean='mean',
    median='median',
    max = 'max',
    min = 'min'
).reset_index()

summary_df

,type,mean,median,max,min
0,Leavers,792.093960,363.0,3697,19
1,Stayers,1660.045706,1258.0,7888,201
